# Objective:
## This code provides a strategy to trade a FX currency pair baed on its historical market data. 

### First, we organized and clean the data using following parameters in MongoDB and SQLite
#### a. Hourly basis
#### b. Same starting date
#### c. Same ending date
#### d. Hourly price (VWAP)
#### e. Hourly liquidity (number of transactions)

#### For every 6 hours , aggregate data into following parameters and generate a csv file:
#### a. Timestamp
#### b. VWAP price
#### c. Liquidity (average number of transactions per hour)
#### d. Volatility 
#### e. Max 
#### f. Min 
#### g. FD

### Second, we use the 70-30 train-test ratio to classify each currency pair over multiple regressions and predict whether the currency pair is forcastable or not base on (MAE,MSE,RMSE,R2,RMSLE)
#### For our forecastable check, we determine when R2 larger than 0.9, MAE, MSE, RMSE, RMSLE smaller than 0.05, then the currency pair is forcastable. Otherwise, the currency pair is not forecastble.

### Third, 


In [ ]:
# Import the necessary libraries and modules from the oandapyV20 package
import oandapyV20
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.pricing as pricing
from oandapyV20.contrib.requests import MarketOrderRequest
from oandapyV20 import API
from oandapyV20.exceptions import V20Error

# Import additional libraries
import pandas as pd
import datetime as dt
import time
from pymongo import MongoClient


In [ ]:
# Set the OANDA account details including the API access token and account ID
access_token = "c6c6c6dc15a9e9639ff4c114d39c1023-c909e030e5acec24e6bde38edf93fbce"
accountID = "101-001-25476922-003"

# Use the OANDA API access token and account ID to create an instance of the oandapyV20.API object
try:
    client = oandapyV20.API(access_token=access_token, environment="practice")
# If an error occurs during the creation of the API object, print the error message
except V20Error as e:
    print("Error: {}".format(e))

In [ ]:
# Set the necessary order parameters
instrument = "AUD/JPY" # Identify the instrument in str format(long currency)
#instrument = "AUD/CHF" # Identify the instrument in str format(short currency)
order_quantity = 1000 # start with 1k units
total_order_quantity = 100000 # the total order quantity
current_executed_quantity = 0 # keep track of how many units have been executed so far
non_executed_quantity = 0 # keep track of the units that were not executed in the previous window
average_execution_price = 0 # initialize the average execution price to 0
total_average_execution_price = 0 # initialize the total average execution price to 0

# setting the required time frames
first_execution_start_time = dt.datetime.now().replace(hour=15, minute=00, second=0, microsecond=0)
first_execution_end_time = dt.datetime.now().replace(hour=17, minute=00, second=0, microsecond=0)

second_execution_start_time = dt.datetime.now().replace(hour=19, minute=00, second=0, microsecond=0)
second_execution_end_time = dt.datetime.now().replace(hour=22, minute=00, second=0, microsecond=0)

third_execution_start_time = dt.datetime.now().replace(hour=23, minute=00, second=0, microsecond=0)
third_execution_end_time = dt.datetime.now().replace(hour=1, minute=00, second=0, microsecond=0) + dt.timedelta(days=1) # add one day to the end time

fourth_execution_start_time = dt.datetime.now().replace(hour=3, minute=00, second=0, microsecond=0) + dt.timedelta(days=1) # add one day to the start time
fourth_execution_end_time = dt.datetime.now().replace(hour=6, minute=00, second=0, microsecond=0) + dt.timedelta(days=1) # add one day to the end time

closing_30_min_start = dt.datetime.now().replace(hour=6, minute=00, second=0, microsecond=0) + dt.timedelta(days=1) # add one day to the end time
closing_30_min_end = dt.datetime.now().replace(hour=6, minute=30, second=0, microsecond=0) + dt.timedelta(days=1) # add one day to the end time

closing_60_min_start = dt.datetime.now().replace(hour=6, minute=30, second=0, microsecond=0) + dt.timedelta(days=1) # add one day to the end time
closing_60_min_end = dt.datetime.now().replace(hour=7, minute=00, second=0, microsecond=0) + dt.timedelta(days=1) # add one day to the end time


In [ ]:
# Define a function to execute long (buy) orders with a given quantity and instrument
def execute_long_order(quantity, trades):
    # create a market order request
    mo = MarketOrderRequest(
        instrument=instrument,
        units=quantity
    )
    # Send the market order request to the API using the `orders.OrderCreate` method
    # and store the response in the `response` variable
    r = orders.OrderCreate(accountID, data=mo.data)
    response = client.request(r)
    # If the order is filled, append the trade details to the `trades` list
    if "orderFillTransaction" in response:
        trades.append({
            "Timestamp": dt.datetime.now(), 
            "Order_ID": float(response["orderFillTransaction"]["orderID"]),
            "Instrument": instrument,
            "Price": float(response["orderFillTransaction"]["price"]),
            "Quantity": float(response["orderFillTransaction"]["units"])
        })
    else:
        # If the order is not filled, handle the case where it is canceled or no cancel transaction is found
        if "orderCancelTransaction" in response:
            print("Order canceled due to:", response["orderCancelTransaction"]["reason"])
        else:
            print("No cancel transaction found.")

In [ ]:
# Define a function to execute short (sell) orders with a given quantity and instrument
def execute_short_order(quantity, trades):
    # create a market order request
    mo = MarketOrderRequest(
        instrument=instrument,
        units=-quantity
    )
    # Send the market order request to the API using the `orders.OrderCreate` method
    # and store the response in the `response` variable
    r = orders.OrderCreate(accountID, data=mo.data)
    response = client.request(r)
    # If the order is filled, append the trade details to the `trades` list
    if "orderFillTransaction" in response:
        trades.append({
            "Timestamp": dt.datetime.now(), 
            "Order_ID": float(response["orderFillTransaction"]["orderID"]),
            "Instrument": instrument,
            "Price": float(response["orderFillTransaction"]["price"]),
            "Quantity": float(response["orderFillTransaction"]["units"])
        })
    else:
        # If the order is not filled, handle the case where it is canceled or no cancel transaction is found
        if "orderCancelTransaction" in response:
            print("Order canceled due to:", response["orderCancelTransaction"]["reason"])
        else:
            print("No cancel transaction found.")

In [ ]:
# Define a function to get price that can be used to compare with the average price before execution takes place
def get_rate():
    price_request = pricing.PricingInfo(accountID=accountID, params={'instruments': instrument})
    response = client.request(price_request)
    return float(response['prices'][0]['bids'][0]['price'])

In [ ]:
# define a function to store the the ttl executed units, average price and the total non-executed value in Mongo DB
def store_to_DB(df):
       
    # create a pymongo client
    client = MongoClient('mongodb://localhost:27017/')

    # select a database and collection
    db = client['final_project_long_result']
    #db = client['final_project_short_result']
    collection = db['AUD_JPY']
    #collection = db['AUD_CHF']

    # convert dataframe to dictionary
    records = df.to_dict('records')

    # insert records into the collection
    collection.insert_many(records)

    # close the pymongo client
    client.close()

# First Trading Period

In [ ]:
# Execute the first trading when the current time reached our designed time
while dt.datetime.now() < first_execution_start_time:
    time.sleep(1)

batch1_data = []  # create a list to store the prices
while dt.datetime.now() >= first_execution_start_time and dt.datetime.now() <= first_execution_end_time: # Loop until the first execution end time (2h) is reached
    for i in range(20): # 20 iterations to execute 1k units every 6 minutes for the 2 hours range
        # If the current executed quantity is in the range of 20% of the total quantity, execute the order
        if current_executed_quantity < total_order_quantity * 0.2:       
            response = execute_long_order(order_quantity, batch1_data)
            #response = execute_short_order(order_quantity, batch1_data)
            current_executed_quantity += order_quantity # Update executed quantity
        else:
            break
        time.sleep(6) # wait for 6 minutes between orders
        print("Successful execution for", i, "the interval")

In [ ]:
# Create a Pandas Dataframe to store the list of trade information previously collected in the batch1_data
df1 = pd.DataFrame(batch1_data)
# Calculate the average execution price for the orders executed in the first trading period
average_execution_price1 = df1['Price'].sum() / len(df1)

# Create the Pandas DataFrame with the desired columns
average_df1 = pd.DataFrame(columns=['average', 'executed so far', 'non-executed'])

for i in range(20): # Max 20 trades
    if i == 0:
        # Create a new row as a dictionary
        new_row = {'average': df1.iloc[i]['Price'], 'executed so far': abs(df1.iloc[i]['Quantity']), 'non-executed': 20000} #100,000 K *0.2

        # Append the new row to the DataFrame
        average_df1 = average_df1.append(new_row, ignore_index=True)
    else:
        # Create a new row as a dictionary
        new_row = {'average': (average_df1.iloc[i-1]['average']*average_df1.iloc[i-1]['executed so far']+df1.iloc[i]['Price']*abs(df1.iloc[i]['Quantity']))/(average_df1.iloc[i-1]['executed so far']+abs(df1.iloc[i]['Quantity'])), 'executed so far': abs(-average_df1.iloc[i-1]['executed so far']+df1.iloc[i]['Quantity']), 'non-executed': 20000-abs(-average_df1.iloc[i-1]['executed so far']+df1.iloc[i]['Quantity'])}

        # Append the new row to the DataFrame
        average_df1 = average_df1.append(new_row, ignore_index=True)
average_df1

# Second Trading Period

In [ ]:
# Execute the second trading when the current time reached our designed time
while dt.datetime.now() < second_execution_start_time:
    time.sleep(1)

batch2_prices = []  # create a list to store the prices

while dt.datetime.now() >= second_execution_start_time and dt.datetime.now() <= second_execution_end_time:
    for i in range(30): # 30 iterations to execute 1k units every 6 minutes for the 3 hours range
        # If the current executed quantity is in the range of 50% of the total quantity, execute the order
        if current_executed_quantity < total_order_quantity * 0.5:
            if get_rate() >= average_execution_price1: # Long
            #if get_rate() <= average_execution_price1: # Short
                execute_long_order(order_quantity, batch2_prices)
                #execute_short_order(order_quantity, batch2_prices)
                current_executed_quantity += order_quantity # Update executed quantity
                
            else:
                print("Price is more than average price")
        else:
            break
        time.sleep(6) # wait for 6 minutes between orders
        print("Successful execution for", i, "th interval")

In [ ]:
# convert the list of prices into a DataFrame
df2 = pd.DataFrame(batch2_prices)
df2
# concatenate the two dataframes vertically
new_df = pd.concat([df1, df2], axis=0)
# calculate the total average execution price
average_execution_price2 = new_df['Price'].sum() / len(new_df)
average_execution_price2

# Create the Pandas DataFrame with the desired columns
average_df1 = pd.DataFrame(columns=['average', 'executed so far', 'non-executed'])

for i in range(20): # Max 30 trades
    if i == 0:
        # Create a new row as a dictionary
        new_row = {'average': df1.iloc[i]['Price'], 'executed so far': abs(df1.iloc[i]['Quantity']), 'non-executed': 20000} #100,000 K *0.2

        # Append the new row to the DataFrame
        average_df1 = average_df1.append(new_row, ignore_index=True)
    else:
        # Create a new row as a dictionary
        new_row = {'average': (average_df1.iloc[i-1]['average']*average_df1.iloc[i-1]['executed so far']+df1.iloc[i]['Price']*abs(df1.iloc[i]['Quantity']))/(average_df1.iloc[i-1]['executed so far']+abs(df1.iloc[i]['Quantity'])), 'executed so far': abs(-average_df1.iloc[i-1]['executed so far']+df1.iloc[i]['Quantity']), 'non-executed': 20000-abs(-average_df1.iloc[i-1]['executed so far']+df1.iloc[i]['Quantity'])}

        # Append the new row to the DataFrame
        average_df1 = average_df1.append(new_row, ignore_index=True)
average_df1

In [ ]:
non_executed_quantity2 = total_order_quantity * 0.5 - current_executed_quantity

# Third Trading Period

In [ ]:
# Execute the third trading when the current time reached our designed time
while dt.datetime.now() < third_execution_start_time:
    time.sleep(1)

batch3_prices = []  # create a list to store the prices
order_quantity_list = [] # create a list to store the order quantity

while dt.datetime.now() >= third_execution_start_time and dt.datetime.now() <= third_execution_end_time:
    # recalculate the number of units to be executed
    order_quantity = int((non_executed_quantity2 + (total_order_quantity * 0.2)) / 20)
    for i in range(20): # 20 iterations to execute updated units every 6 minutes for the 2 hours range
        # If the current executed quantity is in the range of 70% of the total quantity, execute the order
        if current_executed_quantity < total_order_quantity * 0.7:
            if get_rate() >= average_execution_price2: # Long
            #if get_rate() <= average_execution_price2: # Short
                response = execute_long_order(order_quantity, batch3_prices)
                #response = execute_short_order(order_quantity, batch3_prices)
                current_executed_quantity += order_quantity # Update executed quantity
            else:
                print("Price is more than average price")
        else:
            break
        time.sleep(6) # wait for 6 minutes between orders
        print("Successful execution for", i, "th interval")

In [ ]:
# convert the list of prices into a DataFrame
df3 = pd.DataFrame(batch3_prices)
df3
# concatenate the two dataframes vertically
updated_df = pd.concat([new_df, df3], axis=0)
# calculate the total average execution price
average_execution_price3 = updated_df['Price'].sum() / len(updated_df)

In [ ]:
non_executed_quantity3 = total_order_quantity * 0.7 - current_executed_quantity

# Fourth Trading Period

In [ ]:
# Execute the fourth trading when the current time reached our designed time
while dt.datetime.now() < fourth_execution_start_time:
    time.sleep(1)

batch4_prices = []  # create a list to store the prices

while dt.datetime.now() >= fourth_execution_start_time and dt.datetime.now() <= fourth_execution_end_time:
    # recalculate the number of units to be executed
    order_quantity = int((non_executed_quantity3 + (total_order_quantity * 0.3)) / 30)
    for i in range(30): # 30 iterations to execute updated units every 6 minutes for the 3 hours range
        # If the current executed quantity is in the range of the total quantity, execute the order
        if current_executed_quantity < total_order_quantity:
            if get_rate() >= average_execution_price3: # Long
            #if get_rate() <= average_execution_price3: # Short
                response = execute_long_order(order_quantity, batch4_prices)
                #response = execute_short_order(order_quantity, batch4_prices)
                current_executed_quantity += order_quantity # Update executed quantity
            else:
                print("Price is more than average price")
        else:
            break
        time.sleep(6) # wait for 6 minutes between orders
        print("Successful execution for", i, "th interval")

In [ ]:
# convert the list of prices into a DataFrame
df4 = pd.DataFrame(batch4_prices)
df4
# concatenate the two dataframes vertically
final_df = pd.concat([updated_df, df4], axis=0)
# calculate the total average execution price
average_execution_price4 = final_df['Price'].sum() / len(final_df)

In [ ]:
non_executed_quantity_final = total_order_quantity - current_executed_quantity

# Closing Order

In [ ]:
# calculate the total average execution price
average_execution_price_ttl = final_df['Price'].sum() / len(final_df)
average_execution_price_ttl

In [ ]:
# Closing in 30 minutes
while dt.datetime.now() < closing_30_min_start:
    time.sleep(1)

# Check if the current time is within the closing 30-minute window
if closing_30_min_end > dt.datetime.now() > closing_30_min_start: 
    # Calculate the order quantity as 50% of the remaining non-executed quantity
    order_quantity = non_executed_quantity_final * 0.5
    # Create a list to store the prices for this order
    closing_30_data = []

    # Check if the current executed quantity is less than the total order quantity
    if current_executed_quantity < total_order_quantity:
        # Check if the current market rate is lower or equal to the average execution price
        if get_rate() >= average_execution_price_ttl: # Long
        #if get_rate() <= average_execution_price_ttl: # Short
            # Execute the order and store the trade data in the closing_30_data list
            response = execute_long_order(order_quantity, closing_30_data)
            #response = execute_short_order(order_quantity, closing_30_data)
            # Increment the current executed quantity by the order quantity
            current_executed_quantity += order_quantity # Update executed quantity
            print("Successful execution for 50% of total non-executed quantity")
        else:
            print("Price is less than average price") # Long
            print("Price is more than average price") # Short
    # If the current executed quantity is greater or equal to the total order quantity
    elif current_executed_quantity >= total_order_quantity:
        print("All orders have been executed.")

In [ ]:
# convert the list of prices into a DataFrame
df_30_close = pd.DataFrame(closing_30_data)

# concatenate the two dataframes vertically
final_df_30_close = pd.concat([final_df, df_30_close], axis=0)

# calculate new average price after the 30 minute closing activity
average_execution_price_30_close = final_df_30_close['Price'].sum() / len(final_df_30_close)

In [ ]:
# Closing in 60 minutes
while dt.datetime.now() < closing_60_min_start:
    time.sleep(1)

if closing_60_min_end > dt.datetime.now() > closing_60_min_start :
#if dt.datetime.now().replace(hour=23, minute=23, second=0, microsecond=0) > dt.datetime.now() > dt.datetime.now().replace(hour=23, minute=22, second=0, microsecond=0) :
    order_quantity = non_executed_quantity_final*0.5
    closing_60_data = []
    
    if len(closing_30_data) > 0:
        if get_rate() >= average_execution_price_30_close: # Long
        #if get_rate() <= average_execution_price_30_close: # Short
            response = execute_long_order(order_quantity, closing_60_data)
            #response = execute_short_order(order_quantity, closing_60_data)
            print("Successful buy for the remaining 50% of total non executed quantity") # Long
            #print("Successful sell for the remaining 50% of total non executed quantity") # Short
        else:
            response = execute_long_order(order_quantity,closing_60_data) # Long
            #response = execute_short_order(order_quantity,closing_60_data) # Short
            print("Successful buy for the remaining 50% of total non executed quantity") # Long
            #print("Successful sell for the remaining 50% of total non executed quantity") # Short

    elif len(closing_30_data) == 0:
        if get_rate() < average_execution_price_30_close: # Long
        #if get_rate() > average_execution_price_30_close: # Short
            target_quantity = current_executed_quantity - 20000
            total_buy_quantity = 0
            buying_quantity = 1000
            while total_buy_quantity < target_quantity:
                response = execute_short_order(buying_quantity, closing_60_data) # Long
                #response = execute_long_order(buying_quantity, closing_60_data) # Short
                total_buy_quantity += buying_quantity
                print("Successfully execution for selling") # Long
                #print("Successfully execution for buying") # Short
                time.sleep(60) # wait for 1 minute before executing the next order

            print("Target quantity reached.")
            
        else:
            target_quantity = 20000
            total_buy_quantity = 0
            buying_quantity = 1000
            while buying_quantity < target_quantity:
                response = execute_long_order(buying_quantity, closing_60_data) # Long
                response = execute_short_order(buying_quantity, closing_60_data) # Short
                total_buy_quantity += buying_quantity
                
                print("Successfully execution for selling") # Long
                #print("Successfully execution for buying") # Short
                time.sleep(60) # wait for 1 minute before executing the next order

            print("Target quantity reached.")


# Store data in the MongoDB

In [ ]:
store_to_DB(final_df)